In [1]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# HugeCTR demo on Movie lens data

## Overview

HugeCTR is a recommender specific framework which is capable of distributed training across multiple GPUs and nodes for Click-Through-Rate (CTR) estimation. It is a component of NVIDIA [Merlin](https://developer.nvidia.com/nvidia-merlin#getstarted), which is a framework accelerating the entire pipeline from data ingestion and training to deploying GPU-accelerated recommender systems.

### Learning objectives

This notebook demonstrates the steps for training a deep learning recommender model (DLRM) on the movie lens 20M [dataset](https://grouplens.org/datasets/movielens/20m/). We will walk you through the process of data preprocessing, train a DLRM model with HugeCTR, then using the movie embedding to answer item similarity queries.

## Content
1. [Pre-requisite](#1)
1. [Data download and preprocessing](#2)
1. [HugeCTR DLRM training](#3)
1. [Answer item similarity with DLRM embedding](#4)


<a id="1"></a>
## 1. Pre-requisite


### 1.1 Docker containers
Please make sure that you have started the notebook inside the running NGC docker container: `nvcr.io/nvidia/merlin/merlin-training:0.5`. The HugeCTR Python interface have been installed to the system path `/usr/local/hugectr/lib/`. Besides, this system path is added to the environment variable `PYTHONPATH`, which means that you can use the HugeCTR Python interface within the docker container environment.

### 1.2 Hardware
This notebook requires a Pascal, Volta, Turing, Ampere or newer GPUs, such as P100, V100, T4 or A100. 

In [2]:
!nvidia-smi

Tue Apr 27 05:03:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.34       Driver Version: 455.34       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-PCIE-40GB      On   | 00000000:01:00.0 Off |                    0 |
| N/A   28C    P0    29W / 250W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  A100-PCIE-40GB      On   | 00000000:24:00.0 Off |                    0 |
| N/A   

<a id="2"></a>
## 2. Data download and preprocessing

We first install a few extra utilities for data preprocessing.

In [3]:
print("Downloading and installing 'tqdm' package.")
!pip3 -q install torch tqdm

print("Downloading and installing 'unzip' command")
!conda install -y -q -c conda-forge unzip

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda/envs/merlin

  added / updated specs:
    - unzip


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    unzip-6.0                  |       h7f98852_2         143 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         143 KB

The following NEW packages will be INSTALLED:

  unzip              conda-forge/linux-64::unzip-6.0-h7f98852_2


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


Next, we download and unzip the movie lens 20M [dataset](https://grouplens.org/datasets/movielens/20m/).

In [4]:
print("Downloading and extracting 'Movie Lens 20M' dataset.")
!wget -nc http://files.grouplens.org/datasets/movielens/ml-20m.zip -P data -q --show-progress
!unzip -n data/ml-20m.zip -d data
!ls ./data

Archive:  data/ml-20m.zip
hugeCTR  ml-20m  ml-20m.zip  test.parquet  train.parquet  valid.parquet


### Movie lens data preprocessing

In [5]:
import pandas as pd
import torch
import tqdm

In [6]:
MIN_RATINGS = 20
USER_COLUMN = 'userId'
ITEM_COLUMN = 'movieId'

Next, we read the data into a Pandas dataframe, and encode userID and itemID with integers.

In [7]:
df = pd.read_csv('./data/ml-20m/ratings.csv')
print("Filtering out users with less than {} ratings".format(MIN_RATINGS))
grouped = df.groupby(USER_COLUMN)
df = grouped.filter(lambda x: len(x) >= MIN_RATINGS)

print("Mapping original user and item IDs to new sequential IDs")
df[USER_COLUMN], unique_users = pd.factorize(df[USER_COLUMN])
df[ITEM_COLUMN], unique_items = pd.factorize(df[ITEM_COLUMN])

nb_users = len(unique_users)
nb_items = len(unique_items)

print("Number of users: %d\nNumber of items: %d"%(len(unique_users), len(unique_items)))

# Save the mapping to do the inference later on
import pickle
with open('./mappings.pickle', 'wb') as handle:
    pickle.dump({"users": unique_users, "items": unique_items}, handle, protocol=pickle.HIGHEST_PROTOCOL)

Filtering out users with less than 20 ratings
Mapping original user and item IDs to new sequential IDs
Number of users: 138493
Number of items: 26744


Next, we split the data into a train and test set, the last movie each user has recently seen will be used for the test set.

In [8]:
# Need to sort before popping to get the last item
df.sort_values(by='timestamp', inplace=True)
    
# clean up data
del df['rating'], df['timestamp']
df = df.drop_duplicates() # assuming it keeps order

# now we have filtered and sorted by time data, we can split test data out
grouped_sorted = df.groupby(USER_COLUMN, group_keys=False)
test_data = grouped_sorted.tail(1).sort_values(by=USER_COLUMN)
# need to pop for each group
train_data = grouped_sorted.apply(lambda x: x.iloc[:-1])

In [9]:
train_data['target']=1
test_data['target']=1
train_data.head()

,userId,movieId,target
20,0,20,1
19,0,19,1
86,0,86,1
61,0,61,1
23,0,23,1


Since the movie lens data contains only positive examples, let us first define an utility function to generate negative samples.

In [10]:
class _TestNegSampler:
    def __init__(self, train_ratings, nb_users, nb_items, nb_neg):
        self.nb_neg = nb_neg
        self.nb_users = nb_users 
        self.nb_items = nb_items 

        # compute unique ids for quickly created hash set and fast lookup
        ids = (train_ratings[:, 0] * self.nb_items) + train_ratings[:, 1]
        self.set = set(ids)

    def generate(self, batch_size=128*1024):
        users = torch.arange(0, self.nb_users).reshape([1, -1]).repeat([self.nb_neg, 1]).transpose(0, 1).reshape(-1)

        items = [-1] * len(users)

        random_items = torch.LongTensor(batch_size).random_(0, self.nb_items).tolist()
        print('Generating validation negatives...')
        for idx, u in enumerate(tqdm.tqdm(users.tolist())):
            if not random_items:
                random_items = torch.LongTensor(batch_size).random_(0, self.nb_items).tolist()
            j = random_items.pop()
            while u * self.nb_items + j in self.set:
                if not random_items:
                    random_items = torch.LongTensor(batch_size).random_(0, self.nb_items).tolist()
                j = random_items.pop()

            items[idx] = j
        items = torch.LongTensor(items)
        return items

Next, we generate the negative samples for training.

In [11]:
sampler = _TestNegSampler(df.values, nb_users, nb_items, 500)  # using 500 negative samples
train_negs = sampler.generate()
train_negs = train_negs.reshape(-1, 500)

sampler = _TestNegSampler(df.values, nb_users, nb_items, 100)  # using 100 negative samples
test_negs = sampler.generate()
test_negs = test_negs.reshape(-1, 100)

Generating validation negatives...


100%|██████████| 69246500/69246500 [01:05<00:00, 1059133.29it/s]

Generating validation negatives...


100%|██████████| 13849300/13849300 [00:13<00:00, 1048928.58it/s]


In [12]:
import numpy as np

# generating negative samples for training
train_data_neg = np.zeros((train_negs.shape[0]*train_negs.shape[1],3), dtype=int)
idx = 0
for i in tqdm.tqdm(range(train_negs.shape[0])):
    for j in range(train_negs.shape[1]):
        train_data_neg[idx, 0] = i # user ID
        train_data_neg[idx, 1] = train_negs[i, j] # negative item ID
        idx += 1
    
# generating negative samples for testing
test_data_neg = np.zeros((test_negs.shape[0]*test_negs.shape[1],3), dtype=int)
idx = 0
for i in tqdm.tqdm(range(test_negs.shape[0])):
    for j in range(test_negs.shape[1]):
        test_data_neg[idx, 0] = i
        test_data_neg[idx, 1] = test_negs[i, j]
        idx += 1

100%|██████████| 138493/138493 [01:24<00:00, 1635.21it/s]


In [13]:
train_data_np= np.concatenate([train_data_neg, train_data.values])
np.random.shuffle(train_data_np)

test_data_np= np.concatenate([test_data_neg, test_data.values])
np.random.shuffle(test_data_np)


In [14]:
# HugeCTR expect user ID and item ID to be different, so we use 0 -> nb_users for user IDs and
# nb_users -> nb_users+nb_items for item IDs.
train_data_np[:,1] += nb_users 
test_data_np[:,1] += nb_users 

In [15]:
np.max(train_data_np[:,1])

165236

### Write HugeCTR data files

Next, we will write the data to disk using HugeCTR [Norm](../docs/configuration_file_setup.md#norm) dataset format.

In [16]:
from ctypes import c_longlong as ll
from ctypes import c_uint
from ctypes import c_float
from ctypes import c_int

def write_hugeCTR_data(huge_ctr_data, filename='huge_ctr_data.dat'):
    print("Writing %d samples"%huge_ctr_data.shape[0])
    with open(filename, 'wb') as f:
        #write header
        f.write(ll(0)) # 0: no error check; 1: check_num
        f.write(ll(huge_ctr_data.shape[0])) # the number of samples in this data file
        f.write(ll(1)) # dimension of label
        f.write(ll(1)) # dimension of dense feature
        f.write(ll(2)) # long long slot_num
        for _ in range(3): f.write(ll(0)) # reserved for future use

        for i in tqdm.tqdm(range(huge_ctr_data.shape[0])):
            f.write(c_float(huge_ctr_data[i,2])) # float label[label_dim];
            f.write(c_float(0)) # dummy dense feature
            f.write(c_int(1)) # slot 1 nnz: user ID
            f.write(c_uint(huge_ctr_data[i,0]))
            f.write(c_int(1)) # slot 2 nnz: item ID
            f.write(c_uint(huge_ctr_data[i,1]))

#### Train data

In [17]:
def generate_filelist(filelist_name, num_files, filename_prefix):
    with open(filelist_name, 'wt') as f:
        f.write('{0}\n'.format(num_files));
        for i in range(num_files):
            f.write('{0}_{1}.dat\n'.format(filename_prefix, i))

In [18]:
!rm -rf ./data/hugeCTR
!mkdir ./data/hugeCTR

for i, data_arr in enumerate(np.array_split(train_data_np,10)):
    write_hugeCTR_data(data_arr, filename='./data/hugeCTR/train_huge_ctr_data_%d.dat'%i)

generate_filelist('./data/hugeCTR/train_filelist.txt', 10, './data/hugeCTR/train_huge_ctr_data')

Writing 8910827 samples


100%|██████████| 8910827/8910827 [00:35<00:00, 249365.79it/s]

Writing 8910827 samples


100%|██████████| 8910827/8910827 [00:35<00:00, 250680.48it/s]

Writing 8910827 samples


...

100%|██████████| 8910827/8910827 [00:36<00:00, 247415.09it/s]

Writing 8910827 samples


100%|██████████| 8910827/8910827 [00:35<00:00, 247776.47it/s]

Writing 8910827 samples


100%|██████████| 8910827/8910827 [00:36<00:00, 247428.62it/s]


#### Test data

In [19]:
for i, data_arr in enumerate(np.array_split(test_data_np,10)):
    write_hugeCTR_data(data_arr, filename='./data/hugeCTR/test_huge_ctr_data_%d.dat'%i)
    
generate_filelist('./data/hugeCTR/test_filelist.txt', 10, './data/hugeCTR/test_huge_ctr_data')

Writing 1398780 samples


100%|██████████| 1398780/1398780 [00:05<00:00, 241195.10it/s]

Writing 1398780 samples


100%|██████████| 1398780/1398780 [00:05<00:00, 245859.52it/s]

Writing 1398780 samples


...

100%|██████████| 1398779/1398779 [00:05<00:00, 249214.55it/s]

Writing 1398779 samples


100%|██████████| 1398779/1398779 [00:05<00:00, 250584.19it/s]

Writing 1398779 samples


100%|██████████| 1398779/1398779 [00:05<00:00, 249278.58it/s]

<a id="3"></a>
## 3. HugeCTR DLRM training

In this section, we will train a DLRM network on the augmented movie lens data. First, we write the training Python script.

In [20]:
%%writefile hugectr_dlrm_movielens.py
import hugectr
from mpi4py import MPI
solver = hugectr.CreateSolver(max_eval_batches = 1000,
                              batchsize_eval = 65536,
                              batchsize = 65536,
                              lr = 0.1,
                              warmup_steps = 1000,
                              decay_start = 10000,
                              decay_steps = 40000,
                              decay_power = 2.0,
                              end_lr = 1e-5,
                              vvgpu = [[0]],
                              repeat_dataset = True,
                              use_mixed_precision = True,
                              scaler = 1024)
reader = hugectr.DataReaderParams(data_reader_type = hugectr.DataReaderType_t.Norm,
                                  source = ["./data/hugeCTR/train_filelist.txt"],
                                  eval_source = "./data/hugeCTR/test_filelist.txt",
                                  check_type = hugectr.Check_t.Non,
                                  slot_size_array = [138493 , 26744])
optimizer = hugectr.CreateOptimizer(optimizer_type = hugectr.Optimizer_t.SGD,
                                    update_type = hugectr.Update_t.Local,
                                    atomic_update = True)
model = hugectr.Model(solver, reader, optimizer)
model.add(hugectr.Input(label_dim = 1, label_name = "label",
                        dense_dim = 1, dense_name = "dense",
                        data_reader_sparse_param_array = 
                        [hugectr.DataReaderSparseParam(hugectr.DataReaderSparse_t.Localized, 2, 1, 2)],
                        sparse_names = ["data1"]))
model.add(hugectr.SparseEmbedding(embedding_type = hugectr.Embedding_t.LocalizedSlotSparseEmbeddingHash, 
                            slot_size_array = [138493 , 26744],
                            max_vocabulary_size_per_gpu = 165237,
                            embedding_vec_size = 64,
                            combiner = 0,
                            sparse_embedding_name = "sparse_embedding1",
                            bottom_name = "data1",
                            optimizer = optimizer))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.FusedInnerProduct,
                            bottom_names = ["dense"],
                            top_names = ["fc1"],
                            num_output=64))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.FusedInnerProduct,
                            bottom_names = ["fc1"],
                            top_names = ["fc2"],
                            num_output=128))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.FusedInnerProduct,
                            bottom_names = ["fc2"],
                            top_names = ["fc3"],
                            num_output=64))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Interaction,
                            bottom_names = ["fc3","sparse_embedding1"],
                            top_names = ["interaction1"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.FusedInnerProduct,
                            bottom_names = ["interaction1"],
                            top_names = ["fc4"],
                            num_output=1024))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.FusedInnerProduct,
                            bottom_names = ["fc4"],
                            top_names = ["fc5"],
                            num_output=1024))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.FusedInnerProduct,
                            bottom_names = ["fc5"],
                            top_names = ["fc6"],
                            num_output=512))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.FusedInnerProduct,
                            bottom_names = ["fc6"],
                            top_names = ["fc7"],
                            num_output=256))                                                  
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["fc7"],
                            top_names = ["fc8"],
                            num_output=1))                                                                                           
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.BinaryCrossEntropyLoss,
                            bottom_names = ["fc8", "label"],
                            top_names = ["loss"]))
model.compile()
model.summary()
model.fit(max_iter = 50000, display = 1000, eval_interval = 3000, snapshot = 3000, snapshot_prefix = "./hugeCTR_saved_model_DLRM/")

Overwriting hugectr_dlrm_movielens.py


In [21]:
!rm -rf ./hugeCTR_saved_model_DLRM/
!mkdir ./hugeCTR_saved_model_DLRM/

In [22]:
!CUDA_VISIBLE_DEVICES=0 python3 hugectr_dlrm_movielens.py

====================================================Model Init=====================================================
[27d05h37m43s][HUGECTR][INFO]: Global seed is 1972631303
[27d05h37m44s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: A100-PCIE-40GB
[27d05h37m44s][HUGECTR][INFO]: num of DataReader workers: 12
[27d05h37m44s][HUGECTR][INFO]: num_internal_buffers 1
[27d05h37m45s][HUGECTR][INFO]: num_internal_buffers 1
[27d05h37m45s][HUGECTR][INFO]: max_vocabulary_size_per_gpu_=165237
[27d05h37m45s][HUGECTR][INFO]: All2All Warmup Start
[27d05h37m45s][HUGECTR][INFO]: All2All Warmup End
===================================================Model Compile===================================================
[27d05h37m58s][HUGECTR][INFO]: gpu0 start to init embedding of slot0 , slot_size=138493, key_offset=0, value_index_offset=0
[27d05h37m58s][HUGECTR][INFO]: gpu0 start to init embedding of slot1 , slot_size=26744, key_offset=138493, value_index_offset=138493
[27d05h37m58s][

<a id="4"></a>
## 4. Answer item similarity with DLRM embedding

In this section, we demonstrate how the output of HugeCTR training can be used to carry out simple inference tasks. Specifically, we will show that the movie embeddings can be used for simple item-to-item similarity queries. Such a simple inference can be used as an efficient candidate generator to generate a small set of cadidates prior to deep learning model re-ranking. 

First, we read the embedding tables and extract the movie embeddings.

In [23]:
import struct 
import pickle
import numpy as np

key_type = 'I32' # {'I64', 'I32'}, default is 'I32'
key_type_map = {"I32": ["I", 4], "I64": ["q", 8]}

embedding_vec_size = 64

HUGE_CTR_VERSION = 2.21 # set HugeCTR version here, 2.2 for v2.2, 2.21 for v2.21

if HUGE_CTR_VERSION <= 2.2:
    each_key_size = key_type_map[key_type][1] + key_type_map[key_type][1] + 4 * embedding_vec_size
else:
    each_key_size = key_type_map[key_type][1] + 8 + 4 * embedding_vec_size

In [24]:
embedding_table = [{},{}]

with open('./hugeCTR_saved_model_DLRM/0_sparse_9000.model', 'rb') as file:
    try:
        while True:
            buffer = file.read(each_key_size)
            if len(buffer) == 0:
                break
            if HUGE_CTR_VERSION <= 2.2:
                key, slot_id = struct.unpack("2" + key_type_map[key_type][0], 
                                             buffer[0: 2*key_type_map[key_type][1]])
                values = struct.unpack(str(embedding_vec_size) + "f", buffer[2*key_type_map[key_type][1]: ])
            else:
                key = struct.unpack(key_type_map[key_type][0], buffer[0 : key_type_map[key_type][1]])[0]
                slot_id = struct.unpack("Q", buffer[key_type_map[key_type][1] : key_type_map[key_type][1] + 8])[0]
                values = struct.unpack(str(embedding_vec_size) + "f", buffer[key_type_map[key_type][1] + 8: ])
            if slot_id==0:
                embedding_table[slot_id][key] = values
            elif slot_id==1:
                embedding_table[slot_id][key - 138493] = values 
            else:
                raise(Exception("Slot ID not found - %d"%slot_id))
            
    except BaseException as error:
        print(error)

In [25]:
item_embedding = np.zeros((26744, embedding_vec_size), dtype='float')
for i in range(len(embedding_table[1])):
    item_embedding[i] = embedding_table[1][i]
    

In [26]:
len(embedding_table[1])

26744

### Answer nearest neighbor queries


In [27]:
from scipy.spatial.distance import cdist

def find_similar_movies(nn_movie_id, item_embedding, k=10, metric="euclidean"):
    #find the top K similar items according to one of the distance metric: cosine or euclidean
    sim = 1-cdist(item_embedding, item_embedding[nn_movie_id].reshape(1, -1), metric=metric)
   
    return sim.squeeze().argsort()[-k:][::-1]

In [28]:
with open('./mappings.pickle', 'rb') as handle:
    movies_mapping = pickle.load(handle)["items"]

nn_to_movies = movies_mapping
movies_to_nn = {}
for i in range(len(movies_mapping)):
    movies_to_nn[movies_mapping[i]] = i

import pandas as pd
movies = pd.read_csv("./data/ml-20m/movies.csv", index_col="movieId")


In [29]:
for movie_ID in range(1,10):
    try:
        print("Query: ", movies.loc[movie_ID]["title"], movies.loc[movie_ID]["genres"])

        print("Similar movies: ")
        similar_movies = find_similar_movies(movies_to_nn[movie_ID], item_embedding)

        for i in similar_movies:
            print(nn_to_movies[i], movies.loc[nn_to_movies[i]]["title"], movies.loc[nn_to_movies[i]]["genres"])
        print("=================================\n")
    except Exception as e:
        pass

Query:  Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
Similar movies: 
1 Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
2571 Matrix, The (1999) Action|Sci-Fi|Thriller
457 Fugitive, The (1993) Thriller
50 Usual Suspects, The (1995) Crime|Mystery|Thriller
110 Braveheart (1995) Action|Drama|War
32 Twelve Monkeys (a.k.a. 12 Monkeys) (1995) Mystery|Sci-Fi|Thriller
1210 Star Wars: Episode VI - Return of the Jedi (1983) Action|Adventure|Sci-Fi
150 Apollo 13 (1995) Adventure|Drama|IMAX
592 Batman (1989) Action|Crime|Thriller
589 Terminator 2: Judgment Day (1991) Action|Sci-Fi

Query:  Jumanji (1995) Adventure|Children|Fantasy
Similar movies: 
2 Jumanji (1995) Adventure|Children|Fantasy
3996 Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000) Action|Drama|Romance
4886 Monsters, Inc. (2001) Adventure|Animation|Children|Comedy|Fantasy
2396 Shakespeare in Love (1998) Comedy|Drama|Romance
1206 Clockwork Orange, A (1971) Crime|Drama|Sci-Fi|Thriller
454 Firm, The (199